# 12 - Modelos de Gran Contexto y Multimodales

## Curso de LLMs y Aplicaciones de IA

**Duración estimada:** 2 horas

---

## Índice

1. [Modelos de Gran Contexto (LCM)](#lcm)
2. [LCM vs RAG](#comparacion)
3. [Modelos Multimodales](#multimodal)
4. [CLIP: Imágenes + Texto](#clip)
5. [Aplicaciones prácticas](#aplicaciones)
6. [El futuro de los LLMs](#futuro)

---

## Objetivos de aprendizaje

Al finalizar este notebook, serás capaz de:
- Entender las ventajas de modelos con gran ventana de contexto
- Decidir cuándo usar LCM vs RAG
- Comprender modelos multimodales como CLIP
- Implementar búsqueda de imágenes por texto

<a name="lcm"></a>
## 1. Modelos de Gran Contexto (LCM)

### ¿Qué es la ventana de contexto?

La **ventana de contexto** es el número máximo de tokens que un modelo puede procesar en una sola llamada.

| Modelo | Ventana de contexto |
|--------|--------------------|
| GPT-3.5 | 4K - 16K tokens |
| GPT-4 Turbo | 128K tokens |
| Claude 3 | 200K tokens |
| Gemini 1.5 Pro | 1M tokens |
| Llama 3.1 | 128K tokens |

### ¿Qué permite un contexto grande?

- Procesar documentos largos completos
- Analizar libros enteros
- Mantener conversaciones muy largas
- Razonar sobre múltiples documentos a la vez

In [ ]:
!pip install -q transformers torch tiktoken

In [ ]:
import tiktoken

# Token counter
def count_tokens(text, model="gpt-4"):
    """Count tokens in text for a given model."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except:
        encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))

# Example texts
short_text = "Hola, ¿cómo estás?"
medium_text = "La inteligencia artificial es una rama de la informática que busca crear sistemas capaces de realizar tareas que normalmente requieren inteligencia humana. Esto incluye el aprendizaje, el razonamiento, la percepción y el lenguaje natural." * 10
long_text = medium_text * 50

print(f"Texto corto: {count_tokens(short_text)} tokens")
print(f"Texto medio: {count_tokens(medium_text)} tokens")
print(f"Texto largo: {count_tokens(long_text)} tokens")

<a name="comparacion"></a>
## 2. LCM vs RAG

### Comparación

| Aspecto | Gran Contexto (LCM) | RAG |
|---------|---------------------|-----|
| **Acceso a info** | Todo en el prompt | Recuperación selectiva |
| **Persistencia** | No (se pierde) | Sí (vector store) |
| **Costo** | Alto (muchos tokens) | Menor |
| **Latencia** | Mayor | Menor |
| **Precisión** | Puede diluirse | Enfocada |
| **Actualización** | Requiere re-enviar | Fácil añadir docs |

### ¿Cuándo usar cada uno?

**Usa LCM cuando:**
- El documento cabe en la ventana
- Necesitas razonamiento global
- La información está interconectada

**Usa RAG cuando:**
- Tienes muchos documentos
- La información cambia frecuentemente
- Necesitas citaciones precisas
- El costo es importante

In [ ]:
# Example: Token cost estimation
def estimate_cost(tokens, model="gpt-4-turbo"):
    """Estimate API cost for different models."""
    costs = {
        "gpt-4-turbo": 0.01 / 1000,  # $0.01 per 1K input tokens
        "gpt-3.5-turbo": 0.0005 / 1000,
        "claude-3-opus": 0.015 / 1000,
    }
    return tokens * costs.get(model, 0.01/1000)

# Compare costs
doc_tokens = 50000  # 50K tokens (medium document)

print(f"Documento de {doc_tokens:,} tokens")
print(f"\nCosto LCM (todo el documento):")
for model in ["gpt-4-turbo", "gpt-3.5-turbo", "claude-3-opus"]:
    cost = estimate_cost(doc_tokens, model)
    print(f"  {model}: ${cost:.4f}")

print(f"\nCosto RAG (solo chunks relevantes, ~2000 tokens):")
for model in ["gpt-4-turbo", "gpt-3.5-turbo", "claude-3-opus"]:
    cost = estimate_cost(2000, model)
    print(f"  {model}: ${cost:.4f}")

<a name="multimodal"></a>
## 3. Modelos Multimodales

Los modelos **multimodales** pueden procesar múltiples tipos de datos:
- Texto
- Imágenes
- Audio
- Video

### Ejemplos de modelos multimodales

| Modelo | Modalidades | Uso |
|--------|-------------|-----|
| GPT-4V | Texto + Imágenes | Análisis visual, OCR |
| CLIP | Texto + Imágenes | Búsqueda, clasificación |
| Whisper | Audio → Texto | Transcripción |
| DALL-E | Texto → Imágenes | Generación de imágenes |
| Gemini | Texto + Imágenes + Audio + Video | General |

<a name="clip"></a>
## 4. CLIP: Imágenes + Texto

**CLIP** (Contrastive Language-Image Pre-training) aprende la relación entre imágenes y texto.

### Arquitectura

```
┌─────────────┐     ┌─────────────┐
│   Imagen    │     │   Texto     │
└──────┬──────┘     └──────┬──────┘
       ↓                   ↓
┌──────┴──────┐     ┌──────┴──────┐
│  Encoder    │     │  Encoder    │
│  Visual     │     │  Texto      │
└──────┬──────┘     └──────┬──────┘
       ↓                   ↓
   [Vector]           [Vector]
       └───────┬───────┘
               ↓
        [Similitud]
```

### Aplicaciones

- Búsqueda de imágenes por texto
- Clasificación zero-shot
- Generación de imágenes (guía para DALL-E)

In [ ]:
# Install CLIP
!pip install -q ftfy regex tqdm
!pip install -q git+https://github.com/openai/CLIP.git

In [ ]:
import torch
import clip
from PIL import Image
import requests
from io import BytesIO

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

print(f"CLIP cargado en {device}")

In [ ]:
# Download sample images
def load_image_from_url(url):
    """Load image from URL."""
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

# Sample image URLs (using placeholder service)
image_urls = {
    "dog": "https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/YellowLabradorLooking_new.jpg/1200px-YellowLabradorLooking_new.jpg",
    "cat": "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg",
}

# Load images (may fail if URLs are blocked, that's OK)
try:
    images = {name: load_image_from_url(url) for name, url in image_urls.items()}
    print(f"Imágenes cargadas: {list(images.keys())}")
except Exception as e:
    print(f"No se pudieron cargar imágenes: {e}")
    print("Continuando con ejemplo teórico...")
    images = {}

In [ ]:
# Example: Zero-shot classification with CLIP
def classify_image(image, labels, model, preprocess, device):
    """Classify image using CLIP zero-shot."""
    # Preprocess image
    image_input = preprocess(image).unsqueeze(0).to(device)
    
    # Tokenize labels
    text_inputs = clip.tokenize([f"a photo of a {label}" for label in labels]).to(device)
    
    # Get features
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)
        
        # Calculate similarity
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    
    return {label: prob.item() for label, prob in zip(labels, similarity[0])}

# Test classification
if images:
    labels = ["dog", "cat", "bird", "car"]
    for name, img in images.items():
        results = classify_image(img, labels, model, preprocess, device)
        print(f"\nImagen: {name}")
        for label, prob in sorted(results.items(), key=lambda x: -x[1]):
            print(f"  {label}: {prob:.2%}")
else:
    print("Ejemplo teórico: CLIP puede clasificar imágenes sin entrenamiento específico")
    print("Simplemente comparando embeddings de imagen y texto.")

### Búsqueda de imágenes por texto

In [ ]:
# Example: Image search by text
def search_images_by_text(query, image_features, image_names, model, device):
    """Search images using text query."""
    # Encode query
    text_input = clip.tokenize([query]).to(device)
    
    with torch.no_grad():
        text_features = model.encode_text(text_input)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        # Calculate similarities
        similarities = (100.0 * text_features @ image_features.T).softmax(dim=-1)
    
    results = list(zip(image_names, similarities[0].tolist()))
    return sorted(results, key=lambda x: -x[1])

# Pre-compute image features
if images:
    image_names = list(images.keys())
    image_tensors = torch.cat([preprocess(img).unsqueeze(0) for img in images.values()]).to(device)
    
    with torch.no_grad():
        image_features = model.encode_image(image_tensors)
        image_features /= image_features.norm(dim=-1, keepdim=True)
    
    # Search
    queries = ["a fluffy animal", "a pet that barks", "a feline"]
    
    for query in queries:
        results = search_images_by_text(query, image_features, image_names, model, device)
        print(f"\nQuery: '{query}'")
        for name, score in results:
            print(f"  {name}: {score:.2%}")

<a name="aplicaciones"></a>
## 5. Aplicaciones prácticas

### Casos de uso de modelos multimodales

| Aplicación | Modelo | Descripción |
|------------|--------|-------------|
| OCR inteligente | GPT-4V | Extraer texto de imágenes con contexto |
| Búsqueda visual | CLIP | Buscar productos por imagen |
| Accesibilidad | GPT-4V | Describir imágenes para usuarios ciegos |
| E-commerce | CLIP | Clasificación automática de productos |
| Seguridad | CLIP | Moderación de contenido visual |

<a name="futuro"></a>
## 6. El Futuro de los LLMs

### Tendencias actuales

1. **Ventanas de contexto más grandes**: De 4K a 1M+ tokens
2. **Multimodalidad nativa**: Un modelo para todo
3. **Agentes autónomos**: LLMs que ejecutan tareas complejas
4. **Modelos más pequeños y eficientes**: Phi-3, Llama 3 8B
5. **Fine-tuning accesible**: LoRA, QLoRA
6. **Razonamiento mejorado**: Chain of Thought, Tree of Thought

### Arquitecturas emergentes

- **Mixture of Experts (MoE)**: Múltiples modelos especializados
- **State Space Models**: Mamba, alternativa a Transformers
- **Retrieval-Augmented Models**: RAG integrado en el modelo

## Resumen del Curso

### Lo que hemos aprendido

| Notebook | Temas |
|----------|-------|
| 01 | Embeddings básicos: One-Hot, Word2Vec, GloVe |
| 02 | Transformers: BERT, Sentence Transformers |
| 03 | Ingeniería de Prompts |
| 04 | Chatbots básicos |
| 05 | Vector Stores y Retrieval |
| 06 | Introducción a RAG |
| 07 | Reranking y optimización |
| 08 | Introducción a Agentes |
| 09 | Agentes con LangChain |
| 10 | LangGraph y flujos |
| 11 | RAG Avanzado Agentic |
| 12 | Modelos de contexto y multimodales |

### Recursos para continuar aprendiendo

- [LangChain Documentation](https://python.langchain.com/)
- [Hugging Face Course](https://huggingface.co/learn/nlp-course)
- [LangGraph Tutorials](https://langchain-ai.github.io/langgraph/)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)

### ¡Felicidades por completar el curso!

Ahora tienes las bases para construir aplicaciones de IA con LLMs. El campo evoluciona rápidamente, así que sigue practicando y experimentando.

---

## Referencias

- [CLIP Paper](https://arxiv.org/abs/2103.00020)
- [Long Context Models](https://arxiv.org/abs/2307.03172)
- [Multimodal Models Survey](https://arxiv.org/abs/2306.13549)